In [1]:
luna_path = '/Volumes/solo/ali/'

In [2]:
annotations_path = luna_path + 'csv/train/'


npy_path = luna_path + 'Data/train_mask/lung_mask/'
nodemask = luna_path + 'Data/train_mask/nodule_mask/'
#dst_nodules = luna_path + 'Data/val_mask/'
pics_path = luna_path + 'pic/train/true/'

In [3]:
import numpy as np


import time
import gc
import os
from skimage.measure import label,regionprops
import SimpleITK as sitk
import numpy as np
import csv
import scipy
from glob import glob
import pandas as pd
from scipy import ndimage
from tqdm import tqdm 
import pandas as pd
from libtiff import TIFF3D,TIFF
from scipy import misc
from numpy.random import randint

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def get_filename(file_list, case):
    for f in file_list:
        if case in f:
            return(f)
        
def load_itk(filename):
    itkimage = sitk.ReadImage(filename)
    numpyImage = sitk.GetArrayFromImage(itkimage)
    numpyOrigin = np.array(list(reversed(itkimage.GetOrigin())))
    numpySpacing = np.array(list(reversed(itkimage.GetSpacing())))
    return numpyImage, numpyOrigin, numpySpacing

def world_2_voxel(world_coordinates, origin, spacing):
    stretched_voxel_coordinates = np.absolute(world_coordinates - origin)
    voxel_coordinates = stretched_voxel_coordinates / spacing
    return voxel_coordinates

def voxel_2_world(voxel_coordinates, origin, spacing):
    stretched_voxel_coordinates = voxel_coordinates * spacing
    world_coordinates = stretched_voxel_coordinates + origin
    return world_coordinates

def seq(start, stop, step=1):
    n = int(round((stop - start)/float(step)))
    if n > 1:
        return([start + step*i for i in range(n+1)])
    else:
        return([])

def load_train():
    data_path = src
    folders = [x for x in os.listdir(data_path) if 'subset' in x]
    os.chdir(data_path)
    patients = []
    for i in folders:
        os.chdir(data_path + i)
        #print('Changing folder to: {}'.format(data_path + i))
        patient_ids = [x for x in os.listdir(data_path + i) if '.mhd' in x]
        for id in patient_ids:
            j = '{}/{}'.format(i, id)
            patients.append(j)
    return patients




def merge_img():

    path = pics_path
    current_filelist = glob(path + '*.tif')
    imgdir = TIFF3D.open(path+"a.tif",'w')
    imgarr = []
    for i in range(len(current_filelist)):
        img = TIFF.open(path + str(i) + ".tif")
        imgarr.append(img.read_image())
    imgdir.write_image(imgarr)
    for j in current_filelist:
        os.remove(j)

def predict_on_scan(scan):
    model = unet_model()
    model.load_weights(luna_path + 'Data/model/Unet_fulldatagen.h5')

    num_test = scan.shape[0]
    scan = scan.reshape(num_test, 1, 512, 512)
    imgs_mask_test = np.ndarray([num_test, 1, 512, 512],dtype=np.float32)
    for i in range(num_test):
        imgs_mask_test[i] = model.predict([scan[i:i+1]], verbose=0)[0]
    return imgs_mask_test

# 生成训练jpg图片（Train）

In [5]:
npy_path = luna_path + 'Data/train_mask/lung_mask/'
nodemask = luna_path + 'Data/train_mask/nodule_mask/'
#dst_nodules = luna_path + 'Data/val_mask/'
pics_path = luna_path + 'pic/train/true/'
pics_path_false = luna_path + 'pic/train/false/'

In [6]:
file_list = []
subset_path = os.listdir(npy_path)
for _ in range(len(subset_path)):
    if subset_path[_] != '.DS_Store':
        file_list.append(npy_path + subset_path[_])
        
        
file_list1 = []
subset_path1 = os.listdir(nodemask)
for _ in range(len(subset_path1)):
    if subset_path1[_] != '.DS_Store':
        file_list1.append(nodemask + subset_path1[_])

## 生成true样本

In [7]:
for j in tqdm(range(len(file_list))):
    name = file_list[j][-14:-4]
    aa = np.load(file_list[j])
    bb = np.load(file_list1[j])
    cc = aa
    cc = np.squeeze(cc)
    
    #真结节截图
    labels= label(np.squeeze(bb))
    props = regionprops(labels)
    for i in range(len(props)):
        if props[i]['EquivDiameter'] < 30:
            z = int(props[i]['Centroid'][2])
            y = int(props[i]['Centroid'][1])
            x = int(props[i]['Centroid'][0])
            misc.imsave(pics_path+ str(name) + str('_') + str(i) + '.jpg', cc[x,y-16:y+16,z-16:z+16])
    


            
    
 

100%|██████████| 600/600 [05:32<00:00,  2.05it/s]


## 生成false样本

In [8]:
for j in tqdm(range(len(file_list))):
    name = file_list[j][-14:-4]
    aa = np.load(file_list[j])
    bb = np.load(file_list1[j])
    cc = aa
    cc = np.squeeze(cc)
    aa = np.squeeze(aa)
    #真结节截图
    labels= label(np.squeeze(bb))
    props = regionprops(labels)
    
    rand_x = []
    rand_y = []
    rand_z = []
    for i in range(len(props)):        
        z1,z2 = int(props[i]['Centroid'][2])-32,int(props[i]['Centroid'][2])+32
        y1,y2 = int(props[i]['Centroid'][1])-32,int(props[i]['Centroid'][1])+32
        x1,x2 = int(props[i]['Centroid'][0])-32,int(props[i]['Centroid'][0])+32
        
        for u in randint(0,aa.shape[0],size=32):
            if u>z2 or u<z1:
                rand_z.append(u)
        
        for v in randint(64,aa.shape[1]-64,size=32):
            if v>y2 or v<y1:
                rand_y.append(v)
        
        for w in randint(64,aa.shape[2]-64,size=32):
            if w>x2 or w<x1:
                rand_x.append(w)     
        
        for _ in range (5):
            z = rand_z.pop(1)
            y = rand_y.pop(1)
            x = rand_x.pop(1)
            
            misc.imsave(pics_path_false+ str(name) + str('_') + str(i) + '.jpg', aa[x,y-16:y+16,z-16:z+16])

100%|██████████| 600/600 [05:28<00:00,  1.16it/s]


# 生成验证jpg图片（Val）

In [9]:
npy_path = luna_path + 'Data/val_mask/lung_mask/'
nodemask = luna_path + 'Data/val_mask/nodule_mask/'
#dst_nodules = luna_path + 'Data/val_mask/'
pics_path = luna_path + 'pic/val/true/'
pics_path_false = luna_path + 'pic/val/false/'

In [10]:
file_list = []
subset_path = os.listdir(npy_path)
for _ in range(len(subset_path)):
    if subset_path[_] != '.DS_Store':
        file_list.append(npy_path + subset_path[_])
        
        
file_list1 = []
subset_path1 = os.listdir(nodemask)
for _ in range(len(subset_path1)):
    if subset_path1[_] != '.DS_Store':
        file_list1.append(nodemask + subset_path1[_])

In [11]:
for j in tqdm(range(len(file_list))):
    name = file_list[j][-14:-4]
    aa = np.load(file_list[j])
    bb = np.load(file_list1[j])
    cc = aa
    cc = np.squeeze(cc)
    
    labels= label(np.squeeze(bb))
    props = regionprops(labels)
    for i in range(len(props)):
        if props[i]['EquivDiameter'] < 30:
            z = int(props[i]['Centroid'][2])
            y = int(props[i]['Centroid'][1])
            x = int(props[i]['Centroid'][0])
            misc.imsave(pics_path+ str(name) + str('_') + str(i) + '.jpg', cc[x,y-16:y+16,z-16:z+16])

100%|██████████| 200/200 [01:48<00:00,  2.81it/s]


In [12]:
for j in tqdm(range(len(file_list))):
    name = file_list[j][-14:-4]
    aa = np.load(file_list[j])
    bb = np.load(file_list1[j])
    cc = aa
    cc = np.squeeze(cc)
    aa = np.squeeze(aa)
    #真结节截图
    labels= label(np.squeeze(bb))
    props = regionprops(labels)
    
    rand_x = []
    rand_y = []
    rand_z = []
    for i in range(len(props)):        
        z1,z2 = int(props[i]['Centroid'][2])-32,int(props[i]['Centroid'][2])+32
        y1,y2 = int(props[i]['Centroid'][1])-32,int(props[i]['Centroid'][1])+32
        x1,x2 = int(props[i]['Centroid'][0])-32,int(props[i]['Centroid'][0])+32
        
        for u in randint(0,aa.shape[0],size=32):
            if u>z2 or u<z1:
                rand_z.append(u)
        
        for v in randint(64,aa.shape[1]-64,size=32):
            if v>y2 or v<y1:
                rand_y.append(v)
        
        for w in randint(64,aa.shape[2]-64,size=32):
            if w>x2 or w<x1:
                rand_x.append(w)     
        
        for _ in range (5):
            z = rand_z.pop(1)
            y = rand_y.pop(1)
            x = rand_x.pop(1)
            
            misc.imsave(pics_path_false+ str(name) + str('_') + str(i) + '.jpg', aa[x,y-16:y+16,z-16:z+16])

100%|██████████| 200/200 [01:49<00:00,  2.73it/s]
